In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import argparse
import torch

import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange

from diffmask.models.question_answering_squad import BertQuestionAnsweringSquad

from diffmask.attributions.integrated_gradient import squad_bert_integrated_gradient as integrated_gradient
from diffmask.attributions.schulz import bert_hidden_states_statistics, squad_bert_schulz_explainer
from diffmask.attributions.guan import squad_bert_guan_explainer

from diffmask.utils.plot import print_attributions

In [3]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--gpus", type=str, default="0")
    parser.add_argument("--model", type=str, default="bert-large-uncased-whole-word-masking-finetuned-squad")
    parser.add_argument(
        "--train_filename",
        type=str,
        default="./datasets/squad/train-v1.1_bert-large-uncased-whole-word-masking-finetuned-squad.json",
    )
    parser.add_argument(
        "--val_filename",
        type=str,
        default="./datasets/squad/dev-v1.1_bert-large-uncased-whole-word-masking-finetuned-squad.json",
    )
    parser.add_argument("--batch_size", type=int, default=12)
    parser.add_argument("--seed", type=int, default=0)

    hparams, _ = parser.parse_known_args()
    
    torch.manual_seed(hparams.seed)
    
    os.environ["CUDA_VISIBLE_DEVICES"] = hparams.gpus

In [4]:
device = "cuda:0"
model = BertQuestionAnsweringSquad(hparams).train().to(device)
model.prepare_data()
model.freeze()

In [5]:
question = "Where did the Broncos practice for the Super Bowl ?"
context = """The Panthers used the San Jose State practice facility and \
stayed at the San Jose Marriott . The Broncos practiced at Stanford University \
and stayed at the Santa Clara Marriott ."""

inputs_dict = {
    k: v.to(device)
    for k, v in model.tokenizer.encode_plus(
        question,
        context,
        max_length=384,
        pad_to_max_length=True,
        return_tensors="pt",
    ).items()
}
inputs_dict["mask"] = inputs_dict["attention_mask"]
inputs_dict["start_positions"] = torch.tensor([33], device=device)
inputs_dict["end_positions"] = torch.tensor([34], device=device)
del inputs_dict["attention_mask"]

question = model.tokenizer.tokenize(question)
context = model.tokenizer.tokenize(context)
tokens = ["[CLS]"]  + question + ["[SEP]"]  + context + ["[SEP]"]

In [6]:
i = 410
inputs_dict = {
    "input_ids": model.val_dataset[i][0].unsqueeze(0).to(device),
    "mask": model.val_dataset[i][1].unsqueeze(0).to(device),
    "token_type_ids": model.val_dataset[i][2].unsqueeze(0).to(device),
    "start_positions": (model.val_dataset[i][3].unsqueeze(0).to(device))[...,0],
    "end_positions": (model.val_dataset[i][3].unsqueeze(0).to(device) + 1)[...,0],
}
tokens = (
    ["[CLS]"] 
    + list(model.val_dataset_orig.values())[i]["question"] 
    + ["[SEP]"] 
    + list(model.val_dataset_orig.values())[i]["context"] 
    + ["[SEP]"]
 )
tokens = """
[CLS] Where did the Broncos practice for the Super Bowl ? 
[SEP] The Panthers used the San Jose State practice 
facility and stayed at the San Jose Marriott . The 
Broncos practiced at Stanford University and 
stayed at the Santa Clara Marriott . [SEP]
""".replace("\n", "").split(" ")

In [7]:
attributions_ig = integrated_gradient(
    model,
    inputs_dict,
    hidden_state_idx=0,
    steps=500,
)
attributions_ig = attributions_ig.sum(-1).abs()[0,:inputs_dict["mask"].sum()].cpu()

In [8]:
print_attributions(
    tokens, attributions_ig, special=False
)

Where did the Broncos practice for the Super Bowl ? The Panthers used the San Jose State practice facility and stayed at the San Jose Marriott . The Broncos practiced at Stanford University and stayed at the Santa Clara Marriott .


In [9]:
all_q_z_loc, all_q_z_scale = bert_hidden_states_statistics(model, input_only=True)

In [10]:
attributions_schulz = squad_bert_schulz_explainer(
    model,
    inputs_dict,
    all_q_z_loc=all_q_z_loc,
    all_q_z_scale=all_q_z_scale,
    hidden_state_idx=0,
    steps=500,
    lr=1e-1,
    la=10,
)
attributions_schulz = attributions_schulz[0,:inputs_dict["mask"].sum()].cpu()

In [11]:
print_attributions(
    tokens, attributions_schulz, special=False
)

Where did the Broncos practice for the Super Bowl ? The Panthers used the San Jose State practice facility and stayed at the San Jose Marriott . The Broncos practiced at Stanford University and stayed at the Santa Clara Marriott .


In [12]:
attributions_guan = squad_bert_guan_explainer(
    model,
    inputs_dict,
    hidden_state_idx=0,
    steps=500,
    lr=1e-1,
    la=5
)
attributions_guan = 1 / attributions_guan[0,:inputs_dict["mask"].sum()].cpu()

In [13]:
print_attributions(
    tokens, attributions_guan, special=False
)

Where did the Broncos practice for the Super Bowl ? The Panthers used the San Jose State practice facility and stayed at the San Jose Marriott . The Broncos practiced at Stanford University and stayed at the Santa Clara Marriott .


In [14]:
def gen_attention_masks(n, max_length):
    if n == 0:
        return [[0] * max_length]
    elif n == max_length:
        return [[1] * max_length]
    else:
        return [
            [0] + e for e in gen_attention_masks(n=n, max_length=max_length - 1)] + [
            [1] + e for e in gen_attention_masks(n=n - 1, max_length=max_length - 1)
        ]

In [15]:
logits_start_orig, logits_end_orig = model(**inputs_dict)
answer = [e.argmax().item() for e in model(**inputs_dict)]

for n in range(answer[1] - answer[0] + 1, 5):
    print("Trying N={}".format(n))
    all_mask = torch.tensor(gen_attention_masks(n=n, max_length=44))
    all_mask = all_mask[all_mask[:,answer[0]:answer[1] + 1].bool().all(-1)]
    
    solutions = []
    with torch.no_grad():
        t = tqdm(all_mask.split(128))
        for mask in t:

            logits_start, logits_end = model.net(**{
                "inputs_embeds": model.net.bert.embeddings.word_embeddings(
                    inputs_dict["input_ids"]
                ) * torch.nn.functional.pad(mask.to(device), (0, 384 - 44, 0, 0)).unsqueeze(-1),
                "attention_mask": inputs_dict["mask"],
                "token_type_ids": inputs_dict["token_type_ids"]
            })
            
            logits_start, logits_end, logits_start_orig, logits_end_orig = tuple(
                torch.where(inputs_dict['mask'].bool(), e, torch.full_like(e, -float("inf")))
                for e in (logits_start, logits_end, logits_start_orig, logits_end_orig)
            )

            loss_c = (
                logits_start.argmax(-1) == logits_start_orig.argmax(-1)
            ) & (
                logits_end.argmax(-1) == logits_end_orig.argmax(-1)
            )
            
            solutions += [e for e in mask[loss_c]]
            t.set_postfix(solutions=len(solutions))
    
    if len(solutions) == 0:
        print("No solutions with N={}".format(n))
    else:
        break

Trying N=2



No solutions with N=2
Trying N=3



No solutions with N=3
Trying N=4


In [18]:
for e in solutions:
    print_attributions(tokens, e[:inputs_dict["mask"].sum()].cpu(), special=False)

Where did the Broncos practice for the Super Bowl ? The Panthers used the San Jose State practice facility and stayed at the San Jose Marriott . The Broncos practiced at Stanford University and stayed at the Santa Clara Marriott .
Where did the Broncos practice for the Super Bowl ? The Panthers used the San Jose State practice facility and stayed at the San Jose Marriott . The Broncos practiced at Stanford University and stayed at the Santa Clara Marriott .
Where did the Broncos practice for the Super Bowl ? The Panthers used the San Jose State practice facility and stayed at the San Jose Marriott . The Broncos practiced at Stanford University and stayed at the Santa Clara Marriott .
Where did the Broncos practice for the Super Bowl ? The Panthers used the San Jose State practice facility and stayed at the San Jose Marriott . The Broncos practiced at Stanford University and stayed at the Santa Clara Marriott .
Where did the Broncos practice for the Super Bowl ? The Panthers used the Sa